In [2]:
import librosa
import pandas as pd
import IPython.display as ipd
import numpy as np
import tensorflow as tf
from tensorflow import keras
model = keras.models.load_model('mymodel')
counter=0
print('completed step?')

completed step?


In [34]:
#SKIP FOR NOW
#record audio input
import pyaudio
import wave
chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paFloat32  # 32 bits per sample
channels = 1
fs = 44100  # Record at 44100 samples per second
seconds = 3
filename = "input.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)
frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

ModuleNotFoundError: No module named 'pyaudio'

In [3]:
emotionMap = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
path = 'Train/audio/MaleNeutral.wav'
#load recorded data
X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
#get mel spec
df = pd.DataFrame(columns=['mel_spectrogram'])
spectrogram = librosa.feature.melspectrogram(y=X, sr=sample_rate, n_mels=128,fmax=8000) 
db_spec = librosa.power_to_db(spectrogram)
log_spectrogram = np.mean(db_spec, axis = 0)
df.loc[0] = [log_spectrogram]
df = pd.DataFrame(df['mel_spectrogram'].values.tolist())
df.head()
#df.loc[counter] = [log_spectrogram]
#counter=counter+1

,0,1,2,3,4,5,6,7,8,9,...,232,233,234,235,236,237,238,239,240,241
0,-76.382431,-76.002144,-75.831741,-75.860176,-75.761444,-75.704781,-75.987495,-75.849747,-75.909241,-75.872452,...,-75.98243,-75.843842,-75.743576,-76.12011,-75.903351,-75.858574,-75.996353,-75.947311,-76.35714,-76.382431


In [8]:
#formatting data as required to pass to model for prediction
x_log = df.iloc[:, 3:]
y_log = df.iloc[:,:2]
print(x_log.shape)
mean = np.mean(x_log, axis = 0)
std = np.std(x_log, axis=0)
x_log = (x_log - mean)/std
x_log=np.array(x_log)
y_log=np.array(y_log)
print(x_log.shape)
#log_spectrogram.shape
#std.shape
#print(log_spectrogram)
#print(std)

(1, 239)
(1, 239)


In [5]:
sample = x_log[:,:,np.newaxis]
sample.shape
#std.shape
#sample = log_spectrogram[:,:,np.newaxis]
#sample.shape
model.predict(sample)

array([[nan, nan, nan, nan, nan, nan, nan, nan]], dtype=float32)

In [10]:
#SKIP FOR NOW
def findEmotion(fileName):
    
    emotionMap = {1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}
    finalModel = model
    def featureExtraction(path):
        X, sample_rate = librosa.load(path, res_type='kaiser_fast',duration=3,sr=44100,offset=0.5)
        mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
        return mel
    mel = featureExtraction(fileName)
    return model.predict(mel)

In [11]:
#SKIP FOR NOW
fileName = "Train/audio/MaleNeutral.wav"

ipd.Audio(fileName)

output = findEmotion(fileName)

print("The Predicted Output is : ", output)

ValueError: in user code:

    D:\My Folders\Dev\Anaconda\New folder\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    D:\My Folders\Dev\Anaconda\New folder\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    D:\My Folders\Dev\Anaconda\New folder\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    D:\My Folders\Dev\Anaconda\New folder\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    D:\My Folders\Dev\Anaconda\New folder\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    D:\My Folders\Dev\Anaconda\New fold

In [ ]:
#SKIP FOR NOW
df_dc = pd.get_dummies(df, columns=['ColumnToDummyCode'])
data_url = 'https://github.com/openmundi/world.csv/blob/master/countries(249)_num3.csv'
df = pd.read_csv(data_url, index_col=0)

df.head()

#use top-5 categorical accuracy as a reference hallow layer and BiLSTM layer
#Deep CNN has the best performance in terms of classification accuracy, find LWLRAP score